# Astropy Units, Quantities, and Constants

Astropy includes a powerful framework for units that allows users to attach units to scalars and arrays.  These quantities can be manipulated or combined, while keeping track of the units.

For more information about the features presented below, please see the
[astropy.units](https://docs.astropy.org/en/stable/units/index.html) docs.

Also note that this tutorial assumes you have little or no knowledge of the astropy units docs.  If you're moderately familiar with them and are interested in some more complex examples, you might instead prefer the Astropy tutorial on ["Using Astropy Quantities for astrophysical calculations"](http://learn.astropy.org/Quantities.html).

Table of contents:

* <a href="#Representing-units">Representing units</a>
* <a href="#Composite-units">Composite units</a>
* <a href="#Quantity-objects">Quantity objects</a>
* <a href="#Quantity-attributes">Quantity attributes</a>
* <a href="#Quantity-arithmetic-operations">Quantity arithmetic operations</a>
* <a href="#Combining-Quantities">Combining Quantities</a>
* <a href="#Converting-units">Converting units</a>
* <a href="#Decomposing-units">Decomposing units</a>
* <a href="#Integration-with-Numpy-functions">Integration with NumPy functions</a>
* <a href="#Defining-new-units">Defining new units</a>
* <a href="#Using-physical-constants">Using physical constants</a>
* <a href="#Equivalencies">Equivalencies</a>
* <a href="#Performance-consideration">Performance consideration</a>
* <a href="#Putting-it-all-together:--a-concise-example">Putting it all together: a concise example</a>
* <a href="#Exercises">Exercises</a>

## Representing units

First, we need to import the astropy units subpackage (**`astropy.units`**).  Because we probably want to use units in many expressions, it is most concise to rename the subpackage as **`u`**.  This is the standard convention, but note that this will conflict with any variable called **`u`**:

In [1]:
import astropy.units as u

In [2]:
# We will also import numpy here, which we use later.
# "np" is the common naming standard for this import.
import numpy as np

Units can then be accessed as **`u.<unit>`**.  For example, the meter unit is:

In [8]:
u.m  # u.meter works the same (check aliases)

Unit("m")

Units have docstrings, which give some explanatory text about them:

In [4]:
u.m.__doc__

'meter: base unit of length in SI'

In [5]:
u.pc.__doc__

'parsec: approximately 3.26 light-years.'

and a physical type:

In [6]:
u.m.physical_type

PhysicalType('length')

In [7]:
u.s.physical_type

PhysicalType('time')

Many units also have aliases:

In [9]:
u.m.aliases

['meter']

In [10]:
u.meter

Unit("m")

In [11]:
u.arcsec.aliases

['arcsecond']

In [12]:
u.arcsecond

Unit("arcsec")

SI and cgs units are available by default, but Imperial units require the **`imperial`** prefix:

In [13]:
# This is undefined.
u.inch

AttributeError: module 'astropy.units' has no attribute 'inch'.

In [14]:
# Use this instead.
u.imperial.inch

Unit("inch")

Please see the complete list of [available units](https://astropy.readthedocs.org/en/stable/units/index.html#module-astropy.units.si).

## Composite units

Composite units are created using Python numeric operators (e.g., "`*`" (multiplication), "`/`" (division), and "`**`" (power)).

In [15]:
u.km / u.s

Unit("km / s")

In [16]:
u.imperial.mile / u.h

Unit("mi / h")

In [17]:
(u.eV * u.Mpc) / u.Gyr

Unit("Mpc eV / Gyr")

In [18]:
u.cm**3

Unit("cm3")

In [19]:
u.m / u.kg / u.s**2

Unit("m / (kg s2)")

## ``Quantity`` objects
The most useful feature of units is the ability to attach them to scalars or arrays, creating `Quantity` objects. A `Quantity` object contains both a value and a unit.  The most convenient way to create a `Quantity` object is by multiplying the value with its unit.

In [20]:
3.7 * u.au  # Quantity object

<Quantity 3.7 AU>

An equivalent (but more verbose) way of doing the same thing is to use the `Quantity` object's initializer, as demonstrated below.  In general, the more concise form (above) is preferred, as it is closer to how such a quantity would actually be written in text.  The initalizer form has more options, though, which you can learn about from the [Astropy reference documentation on Quantity](https://docs.astropy.org/en/stable/api/astropy.units.quantity.Quantity.html).

In [21]:
u.Quantity(3.7, unit=u.au)

<Quantity 3.7 AU>

In [22]:
u.Quantity(1 * u.cm, unit=u.m)

<Quantity 0.01 m>

Where quantities really shine is when you make an array `Quantity` object.

In [23]:
# We want to create the composite unit first, hence the parenthesis.
x = [1.2, 6.8, 3.7] * (u.pc / u.year)
x

<Quantity [1.2, 6.8, 3.7] pc / yr>

## `Quantity` attributes

The units and value of a `Quantity` can be accessed separately via the ``value`` and ``unit`` attributes:

In [24]:
q = 5. * u.Mpc
q

<Quantity 5. Mpc>

In [25]:
q.value

5.0

In [26]:
q.unit

Unit("Mpc")

In [27]:
x = [1.2, 6.8, 3.7] * (u.pc / u.year)
x

<Quantity [1.2, 6.8, 3.7] pc / yr>

In [28]:
x.value

array([1.2, 6.8, 3.7])

In [29]:
x.unit

Unit("pc / yr")

## `Quantity` arithmetic operations

"`*`" (multiplication), "`/`" (division), and "`**`" (power) operations can be performed on `Quantity` objects with `float`/`int` values.

In [30]:
q = 3.1 * u.km

In [31]:
q * 2

<Quantity 6.2 km>

In [32]:
q / 2.

<Quantity 1.55 km>

In [33]:
1 / q

<Quantity 0.32258065 1 / km>

In [34]:
q ** 2

<Quantity 9.61 km2>

## Combining Quantities

Quantities can be combined using Python numeric operators:

In [35]:
q1 = 3. * (u.m / u.s)
q1

<Quantity 3. m / s>

In [36]:
q2 = 5. * (u.cm / u.s / u.g**2)
q2

<Quantity 5. cm / (s g2)>

In [37]:
q1 * q2

<Quantity 15. cm m / (g2 s2)>

In [38]:
q1 / q2  # Note the "second" unit cancelled out.

<Quantity 0.6 g2 m / cm>

In [39]:
# Sometimes, one more step needed for "clean" output unit.
# Also see: Decomposing units (below)
(q1 / q2).to(u.g ** 2)

<Quantity 60. g2>

In [40]:
q1 ** 2

<Quantity 9. m2 / s2>

In [41]:
x = [1.2, 6.8, 3.7] * (u.pc / u.year)
x * 3  # Element-wise multiplication.

<Quantity [ 3.6, 20.4, 11.1] pc / yr>

When adding or subtracting quantities, the units must be **compatible** (but not necessarily identical).

In [42]:
# Add two quantities.
(3 * u.m) + (5 * u.m)

<Quantity 8. m>

Here we add two distance quantities that do not have identical units:

In [43]:
(3 * u.km) + (5 * u.cm)

<Quantity 3.00005 km>

In [44]:
# This will fail because the units are incompatible.
(3 * u.km) + (5. * u.km / u.s)

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

## Converting units

Units can be converted to other equivalent units.

In [45]:
q = 2.5 * u.year
q

<Quantity 2.5 yr>

In [46]:
# Convert year to seconds.
q.to(u.s)

<Quantity 78894000. s>

In [47]:
# Convert degree squared to steradian.
(7. * u.deg**2).to(u.sr)

<Quantity 0.00213232 sr>

In [48]:
# Convert mph to kph.
(55. * (u.imperial.mile / u.h)).to(u.km / u.h)

<Quantity 88.51392 km / h>

**Important Note**:  Converting a unit (not a `Quantity`) gives only the scale factor:

In [49]:
u.Msun.to(u.kg)

1.988409870698051e+30

To keep the units, use a `Quantity` (value and unit) object:

In [50]:
(1. * u.Msun).to(u.kg)

<Quantity 1.98840987e+30 kg>

## Decomposing units

The units of a `Quantity` object can be decomposed into a set of base units using the
``decompose()`` method. By default, units will be decomposed to SI unit bases:

In [51]:
q = 8. * (u.cm * u.pc / u.g / u.year**2)
q

<Quantity 8. cm pc / (g yr2)>

In [52]:
q.decompose()

<Quantity 2478.74926276 m2 / (kg s2)>

To decompose into cgs unit bases:

In [53]:
q.decompose(u.cgs.bases)

<Quantity 24787.4926276 cm2 / (g s2)>

In [54]:
u.cgs.bases

{Unit("K"),
 Unit("cd"),
 Unit("cm"),
 Unit("g"),
 Unit("mol"),
 Unit("rad"),
 Unit("s")}

In [55]:
u.si.bases

{Unit("A"),
 Unit("K"),
 Unit("cd"),
 Unit("kg"),
 Unit("m"),
 Unit("mol"),
 Unit("rad"),
 Unit("s")}

Units will not cancel out unless they are identical:

In [56]:
q = 7 * u.m / (7 * u.km)
q

<Quantity 1. m / km>

But they will cancel by using the `decompose()` method:

In [57]:
x = q.decompose()
x  # This is a "dimensionless" Quantity.

<Quantity 0.001>

In [58]:
repr(x.unit)

'Unit(dimensionless)'

## Integration with NumPy functions

Most [NumPy](https://www.numpy.org) functions understand `Quantity` objects:

In [59]:
np.sin(30)  # np.sin assumes the input is in radians.

-0.9880316240928618

In [60]:
np.sin(30 * u.degree)  # Awesome!

<Quantity 0.5>

In [61]:
q = 100 * (u.kg * u.kg)
np.sqrt(q)

<Quantity 10. kg>

In [62]:
x = np.arange(10) * u.km
x

<Quantity [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.] km>

In [63]:
np.mean(x)

<Quantity 4.5 km>

Some NumPy "ufuncs" (universal functions) require dimensionless quantities.

In [64]:
np.log10(4 * u.m)  # This is not possible.

UnitTypeError: Can only apply 'log10' function to dimensionless quantities

In [65]:
np.log10(4 * u.m / (4 * u.km))  # Note that the units cancelled.

<Quantity -3.>

Care needs to be taken with dimensionless units.

For example, passing ordinary values to an inverse trigonometric function gives a result without units:

In [66]:
np.arcsin(1.0)

1.5707963267948966

`u.dimensionless_unscaled` creates a ``Quantity`` with a "dimensionless unit" and therefore gives a result *with* units:

In [67]:
np.arcsin(1.0 * u.dimensionless_unscaled)

<Quantity 1.57079633 rad>

In [68]:
np.arcsin(1.0 * u.dimensionless_unscaled).to(u.degree)

<Quantity 90. deg>

**Important Note:**  In-place array operations do not work with units.

In [69]:
a = np.arange(10.)
a *= 1.0 * u.kg  # In-place operator will fail.

UnitTypeError: Cannot store quantity with dimension resulting from multiply function in a non-Quantity instance.

Assign to a *new* array instead:

In [70]:
a = a * 1.0 * u.kg
a

<Quantity [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.] kg>

Also, Quantities lose their units with some NumPy operations, for example:

* np.append
* np.dot
* np.hstack
* np.vstack
* np.where
* np.choose
* np.vectorize

See [Quantity Known Issues](https://docs.astropy.org/en/stable/known_issues.html#quantities-lose-their-units-with-some-operations) for more details.

## Defining new units

You can also define custom units for something that isn't built in to Astropy.

Let's define a unit called **"sol"** that represents a Martian day.

In [71]:
sol = u.def_unit('sol', 1.0274912510 * u.day)

In [72]:
(1. * u.yr).to(sol)  # 1 Earth year in Martian sol units.

<Quantity 355.47747939 sol>

Now let's define Mark Watney's favorite unit, the [**Pirate-Ninja**](https://en.wikipedia.org/wiki/List_of_humorous_units_of_measurement#Pirate_Ninja):

In [73]:
pirate_ninja = u.def_unit('☠️👤', 1.0 * (u.kW * u.hr / sol))

In [74]:
5.2 * pirate_ninja

<Quantity 5.2 ☠️👤>

In [75]:
# Mars oxygenator power requirement for 6 people.
(44.1 * pirate_ninja).to(u.W)

<Quantity 1788.33639528 W>

## Using physical constants

The [astropy.constants](https://docs.astropy.org/en/stable/constants/index.html) module contains physical constants relevant for astronomy.  They are defined as ``Quantity`` objects using the ``astropy.units`` framework.

Please see the complete list of [available physical constants](https://docs.astropy.org/en/stable/constants/index.html#module-astropy.constants).  Additions are welcome!

In [76]:
from astropy import constants as const

In [77]:
const.G

<<class 'astropy.constants.codata2018.CODATA2018'> name='Gravitational constant' value=6.6743e-11 uncertainty=1.5e-15 unit='m3 / (kg s2)' reference='CODATA 2018'>

In [78]:
const.c

<<class 'astropy.constants.codata2018.CODATA2018'> name='Speed of light in vacuum' value=299792458.0 uncertainty=0.0 unit='m / s' reference='CODATA 2018'>

In [79]:
const.L_sun

<<class 'astropy.constants.iau2015.IAU2015'> name='Nominal solar luminosity' value=3.828e+26 uncertainty=0.0 unit='W' reference='IAU 2015 Resolution B 3'>

Constants are Quantities, thus they can be coverted to other units:

In [80]:
const.L_sun.to(u.erg / u.s)

<Quantity 3.828e+33 erg / s>

Also note that even constants are not constant. `astropy.constants` supports [different versions](https://docs.astropy.org/en/stable/constants/index.html#collections-of-constants-and-prior-versions).

In [81]:
import astropy

In [82]:
astropy.physical_constants.get()

'codata2018'

In [83]:
astropy.astronomical_constants.get()

'iau2015'

In [84]:
from astropy.constants import iau2012
iau2012.L_sun

<<class 'astropy.constants.iau2012.IAU2012'> name='Solar luminosity' value=3.846e+26 uncertainty=5e+22 unit='W' reference="Allen's Astrophysical Quantities 4th Ed.">

In [ ]:
# Changing the entire science state like this after
# imports is currently not possible, see
# https://github.com/astropy/astropy/issues/8781
astropy.astronomical_constants.set('iau2012')

## Equivalencies

Equivalencies can be used to convert quantities that are not strictly the same physical type, but in a specific context are interchangable.  A familiar physics example is the mass-energy equivalency: these are strictly different physical types, but it is often understood that you can convert between the two using $E=mc^2$:

In [85]:
from astropy.constants import m_p  # Proton mass

In [86]:
# This raises an error because mass and energy are different units.
(m_p).to(u.eV)

UnitConversionError: 'kg' (mass) and 'eV' (energy/torque/work) are not convertible

In [87]:
# This succeeds using equivalencies.
(m_p).to(u.MeV, u.mass_energy())

<Quantity 938.27208816 MeV>

This concept extends further in `astropy.units` to include some common practical astronomy situations where the units have no direct physical connection, but it is often useful to have a "quick shorthand."  For example, astronomical spectra are often given as a function of wavelength, frequency, or even energy of the photon.  Suppose you want to find the [Lyman-limit](https://en.wikipedia.org/wiki/Lyman_limit) wavelength:

In [88]:
# This raises an error.
(13.6 * u.eV).to(u.Angstrom)

UnitConversionError: 'eV' (energy/torque/work) and 'Angstrom' (length) are not convertible

Normally, you can convert `u.eV` only to the following units:

In [89]:
u.eV.find_equivalent_units()

Primary name,Unit definition,Aliases
J,m2 kg / s2,"Joule, joule"
Ry,2.17987e-18 m2 kg / s2,rydberg
eV,1.60218e-19 m2 kg / s2,electronvolt
erg,1e-07 m2 kg / s2,


But by using a spectral equivalency, you can also convert `u.eV` to the following units:

In [90]:
u.eV.find_equivalent_units(equivalencies=u.spectral())

Primary name,Unit definition,Aliases
Bq,1 / s,becquerel
Ci,3.7e+10 / s,curie
Hz,1 / s,"Hertz, hertz"
J,m2 kg / s2,"Joule, joule"
Ry,2.17987e-18 m2 kg / s2,rydberg
eV,1.60218e-19 m2 kg / s2,electronvolt
erg,1e-07 m2 kg / s2,
k,100 / m,"Kayser, kayser"
m,irreducible,meter


In [91]:
# Now back to Lyman-limit.
(13.6 * u.eV).to(u.Angstrom, equivalencies=u.spectral())

<Quantity 911.64851789 Angstrom>

Or if you remember the 21cm HI line, but cannot remember the frequency, you could do:

In [92]:
(21. * u.cm).to(u.GHz, equivalencies=u.spectral())

<Quantity 1.42758313 GHz>

To go one step further, the units of a spectrum's *flux* are further complicated by being dependent on the units of the spectrum's "X-axis" (i.e., $f_{\lambda}$ for flux per unit wavelength or $f_{\nu}$ for flux per unit frequency).  `astropy.units` supports this use case, but it is necessary to supply the location in the spectrum where the conversion is done:

In [93]:
q = 1e-18 * (u.erg / u.s / u.cm**2 / u.AA)
q

<Quantity 1.e-18 erg / (Angstrom s cm2)>

In [94]:
q.to(u.uJy, equivalencies=u.spectral_density(1. * u.um))

<Quantity 3.33564095 uJy>

There's a lot of flexibility with equivalencies, including a variety of other useful built-in equivalencies.  So if you want to know more, you might want to check out the [equivalencies narrative documentation](https://docs.astropy.org/en/stable/units/equivalencies.html) or the [astropy.units.equivalencies reference docs](https://docs.astropy.org/en/stable/units/index.html#module-astropy.units.equivalencies).

## Domain-specific units

Some astropy modules define more specific units and equivalencies.

For example, ``astropy.cosmology`` defines units for cosmological redshift and
the Hubble parameter and equivalencies for both.

In [ ]:
import astropy.cosmology.units as cu

cu.__all__

In [ ]:
z = 1100 * cu.redshift
repr(z)

Redshift isn't a "real" unit (so the ``dimensionless_redshift`` equivalency
is turned on by default) but it is very useful for converting between cosmological
distance measures, for example the background temperature. We can do this
in a specific cosmological context:

In [ ]:
from astropy.cosmology import Planck18  # cosmological context

z.to(u.K, equivalencies=cu.redshift_temperature(Planck18))

If you want to know more about cosmology and units, you might want to check out the [astropy.cosmology.units  reference docs](https://docs.astropy.org/en/stable/cosmology/units.html#cosmological-units-and-equivalencies).

# Performance consideration

For more details about performance on big datasets, see [units performance tips](https://docs.astropy.org/en/stable/units/index.html#performance-tips).

When working with a big data array, the `<<` operator can be used to speed up Quantity initialization. Let's consider this array:

In [95]:
big_array = np.random.random(10_000_000)

In [96]:
# Defining the Quantity the simplest way without parenthesis.
%timeit big_array * u.m / u.s / u.kg / u.sr

82.3 ms ± 3.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [97]:
# Defining the Quantity with the * operator and parenthesis.
%timeit big_array * (u.m / u.s / u.kg / u.sr)

20 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [98]:
# Defining the Quantity with the << operator (parenthesis not needed).
%timeit big_array << u.m / u.s / u.kg / u.sr

27.8 µs ± 957 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Replace the ``Quantity`` values below with your slow/fast timing to see how much things sped up on your machine. For example:

In [99]:
((89.4 * u.ms) / (67.1 * u.us)).to(u.dimensionless_unscaled)

<Quantity 1332.33979136>

# Putting it all together:  a concise example

Let's estimate the (circular) orbital speed of the Earth around the Sun using Kepler's Law:

$$v = \sqrt{\frac{G M_{\odot}}{r}}$$

In [100]:
v = np.sqrt(const.G * 1 * u.M_sun / (1 * u.au))
v

<Quantity 8.16963891e-06 m(3/2) solMass(1/2) / (AU(1/2) kg(1/2) s)>

That's a velocity unit... but it sure isn't obvious when you look at it!

Let's use a variety of the available ``Quantity`` methods to get something more sensible:

In [101]:
v.decompose()  # Remember that the default uses SI bases

<Quantity 29784.69182968 m / s>

In [102]:
v.decompose(u.cgs.bases)

<Quantity 2978469.18296769 cm / s>

In [103]:
v.to(u.km / u.s)

<Quantity 29.78469183 km / s>

# Exercises

## Exercise 1

The *James Webb Space Telescope (JWST)* is in a halo orbit around the second Sun-Earth Lagrange (L2) point:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;☀️ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 🌎 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; L2 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *(not to scale)*


L2 is located at a distance from the Earth (opposite the Sun) of approximately:

$$ r \approx R \left(\frac{M_{earth}}{3 M_{sun}}\right) ^{(1/3)} $$

where $R$ is the Sun-Earth distance.

Calculate the Earth-L2 distance in kilometers and miles.

*Hints*:

* $M_{earth}$ and $M_{sun}$ are defined [constants](https://docs.astropy.org/en/stable/constants/#reference-api) 

* The mile unit is defined as ``u.imperial.mile`` (see [imperial units](https://docs.astropy.org/en/v0.2.1/units/index.html#module-astropy.units.imperial))

In [114]:
# Answer here (km)
R = (1 * u.au).to(u.km)
m = ((1 * u.M_earth) / (3 * u.M_sun)) **(1/3)
r = R * m.decompose()
r

<Quantity 1496558.48134108 km>

In [115]:
# Answer here (mile)
r.to(u.imperial.mile)

<Quantity 929918.3278038 mi>

## Exercise 2

The L2 point is about 1.5 million kilometers away from the Earth opposite the Sun.
The total mass of the *James Webb Space Telescope (JWST)* is about 6500 kg.

Using the value you obtained above for the Earth-L2 distance, calculate the gravitational force in Newtons between: 

* *JWST* (at L2) and the Earth
* *JWST* (at L2) and the Sun

*Hint*: the gravitational force between two masses separated by a distance *r* is:

$$ F_g = \frac{G m_1 m_2}{r^2} $$

In [ ]:
# Answer here (JWST and Earth)

In [ ]:
# Answer here (JWST and Sun)

## Exercise 3

Calculate the [Schwarzschild radius](https://en.wikipedia.org/wiki/Schwarzschild_radius) in units of solar radii of the Sgr A*, the Milky Way's supermassive black hole with $M = 4.31 \times 10^6 M_\odot$, given

$$r_\mathrm{s} = \frac{2 G M}{c^2}$$

Also calculate the angular size of the event horizon on the sky in microarcseconds, given the distance to the galactic center $d_{center} = 7.94$ kpc, given

$$\theta = \mathrm{arctan}\frac{2 r_\mathrm{s}}{d_{center}}$$

In [ ]:
# Answer radius here

In [ ]:
# Answer angular size here

## Exercise 4

We can make a very rough estimate of the temperature of material in the vicinity of Sgr A* by assuming hydrostatic equilibrium, so that the thermal energy of the gas balances the gravitational force:

$$ kT \sim GM m_p /R $$

where $m_p$ is the mass of a proton and $R$ is the distance away from the black hole. Using Astropy constants and the properties of Sgr A* described in Example 3, compute the temperature of the gas required to balance the black hole's gravitation pull at 1 million Schwarzschild radii derived above. Use this equation:

$$ T = \frac{G M m_p}{10^6 r_s k} $$

Then use the [Astropy units temperature equivalencies](https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-equivalency) to find the energy of that gas.

In [ ]:
# Answer temperature here

In [ ]:
# Answer energy here